In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

In [2]:
def get_df(file):
    parsed = ET.parse(file)
    root = parsed.getroot()
    
    data = []

    for i, child in enumerate(root):
        data.append(child.attrib)
        
    dfItem = pd.DataFrame.from_records(data).set_index('Id')
    return dfItem

In [6]:
file_directory = '../../../../../Volumes/easystore/DataScience/superuser.com'
file = '/Posts.xml'
file_path = file_directory + file

In [7]:
parsed = ET.parse(file_path)
root = parsed.getroot()

In [8]:
data = []

for i, child in enumerate(root):
    data.append(child.attrib)

#df = pd.DataFrame(data).T  # Write in DF and transpose it
#df.columns = cols  # Update column names

In [6]:
dfItem = pd.DataFrame.from_records(data).set_index('Id')

In [22]:
dfItem.value_counts('Name')[20:50]

Name
Good Answer          11001
Caucus                9820
Enlightened           9528
Excavator             8361
Curious               7258
Good Question         6819
Self-Learner          5871
Announcer             5571
Promoter              5364
Constituent           4566
Organizer             4261
Enthusiast            3168
Guru                  3103
Benefactor            2589
Beta                  1823
Quorum                1704
Peer Pressure         1703
Analytical            1687
Explainer             1664
Mortarboard           1593
Favorite Question     1451
Great Answer          1436
Cleanup               1375
Civic Duty            1278
Suffrage              1120
Taxonomist            1101
Investor              1001
Tag Editor             993
Fanatic                947
Booster                941
dtype: int64

In [9]:
posts = get_df(file_path)

In [33]:
postHistory = get_df(file_path)

In [37]:
comments = get_df(file_path)

In [9]:
users = get_df(file_path)

In [44]:
tags = get_df(file_path)

In [48]:
postLinks = get_df(file_path)

In [61]:
badges = get_df(file_path)

In [63]:
votes = get_df(file_path)

In [51]:
relevant = posts[['PostTypeId', 'CreationDate', 'Score', 'Body', 'Tags', 'OwnerUserId', 'AnswerCount', 'CommentCount', 'ParentId', 'ClosedDate']]
relevant

,PostTypeId,CreationDate,Score,Body,Tags,OwnerUserId,AnswerCount,CommentCount,ParentId,ClosedDate
Id,,,,,,,,,,
1,1,2009-07-15T06:27:46.723,175,<p>A Vista virtual machine I use only has a 10...,<windows><disk-space><winsxs>,1,12,10,NaN,NaN
24,1,2009-07-08T16:32:47.817,7,<p>I used to record important technical meetin...,<audio><audio-recording><camtasia>,NaN,6,0,NaN,NaN
25,2,2009-07-08T16:34:52.867,0,<p>You could use a filter on the recording. A ...,NaN,9637,NaN,0,24,NaN
26,2,2009-07-08T16:59:00.587,0,"<p>Try out something like Audible for Macs, an...",NaN,23242,NaN,0,24,NaN
27,2,2009-07-08T17:15:59.477,1,<p>You definitely need some sort of software t...,NaN,45675,NaN,1,24,NaN
...,...,...,...,...,...,...,...,...,...,...
1607673,1,2020-12-06T04:16:34.360,0,<p>I have always created Virtual Machines on m...,<windows><virtualbox><virtual-machine><azure><...,475817,0,0,NaN,NaN
1607675,1,2020-12-06T04:27:04.807,0,"<p><strong>=VLOOKUP(VLOOKUP(A7,InventoryLog,5,...",<microsoft-excel><worksheet-function>,1248251,0,1,NaN,NaN
1607676,2,2020-12-06T04:34:37.490,0,"<p>If you have Python 3 installed, you could u...",NaN,470014,NaN,0,1315365,NaN


In [56]:
questions = relevant[relevant['PostTypeId'] == '1'].drop(['PostTypeId', 'ParentId'], axis=1)
answers = relevant[relevant['PostTypeId'] == '2'].drop(['PostTypeId', 'AnswerCount', 'ClosedDate', 'Tags'], axis=1)

In [57]:
print('Percentage of questions closed: ' + str(sum(questions.ClosedDate.notna())/questions.shape[0]*100))
print('Percentage of questions with no answers: ' + str(questions[questions['AnswerCount'] == '0'].shape[0]/questions.shape[0]*100))
print('Percentage of comments with a score: ' + str(answers[answers['Score'] > '0'].shape[0]/answers.shape[0]*100))

Percentage of questions closed: 5.9437736831193115
Percentage of questions with no answers: 19.09008297638636
Percentage of comments with a score: 61.93984593944777


We see that there are a lot of questions that never got closed, however, this may be due to reasons other than the question not be satisfied with an answer. However, when we look at the percetnage of questions with no answers, we see 19% of questions never had answers, which means roughly 1 out of 5 questions never got answered, with likely manner more that may have answers that did not satisfy the question. One can also see that roughly 40% of the questions were considered good by someone, but 60% were not. This means we can gather a lot of information about who has been able to help solve a question, but it also shows us that there is a lack of quality answers, and that the SuperUser forum can strive to have higher quality responses in terms of score. We want to build this recommender to be able to have more posts get their questions resolved than we currently have and allow for more quality answers to shwo up for these questions.

In [58]:
# Questions only
questions

,CreationDate,Score,Body,Tags,OwnerUserId,AnswerCount,CommentCount,ClosedDate
Id,,,,,,,,
1,2009-07-15T06:27:46.723,175,<p>A Vista virtual machine I use only has a 10...,<windows><disk-space><winsxs>,1,12,10,NaN
24,2009-07-08T16:32:47.817,7,<p>I used to record important technical meetin...,<audio><audio-recording><camtasia>,NaN,6,0,NaN
29,2009-07-15T07:11:46.370,23,<p>What is the difference between the two supe...,<linux><unix><sudo>,13,9,0,NaN
37,2009-07-15T07:17:13.970,4,"<p>My late 2008, Macbook Pro Unibody machine h...",<mac><crash><boot-camp>,26,3,2,NaN
46,2009-07-15T07:20:00.717,16,<p>Is there any way to configure application t...,<windows><boot><windows-vista><uac><elevation>,21,2,0,NaN
...,...,...,...,...,...,...,...,...
1607669,2020-12-06T03:17:52.090,0,<p>I'm trying to get the &quot;Facebook Page U...,<facebook><advertisements>,930695,0,0,2020-12-06T03:32:19.217
1607671,2020-12-06T03:59:14.313,0,"<p>When I start for Windows settings, such as ...",<windows-10><start-menu><windows-search>,1017832,0,0,NaN
1607673,2020-12-06T04:16:34.360,0,<p>I have always created Virtual Machines on m...,<windows><virtualbox><virtual-machine><azure><...,475817,0,0,NaN


In [59]:
# Answers only
answers

,CreationDate,Score,Body,OwnerUserId,CommentCount,ParentId
Id,,,,,,
25,2009-07-08T16:34:52.867,0,<p>You could use a filter on the recording. A ...,9637,0,24
26,2009-07-08T16:59:00.587,0,"<p>Try out something like Audible for Macs, an...",23242,0,24
27,2009-07-08T17:15:59.477,1,<p>You definitely need some sort of software t...,45675,1,24
34,2009-07-15T07:16:12.363,9,"<p><a href=""http://audacity.sourceforge.net/"" ...",17,1,24
38,2009-07-15T07:17:21.910,1,<p>On systems where the superuser account is d...,55,1,29
...,...,...,...,...,...,...
1607668,2020-12-06T02:59:58.460,0,"<p>These are collapsible styles, you will not ...",1248233,0,1604060
1607670,2020-12-06T03:17:54.600,0,"<p>As <a href=""https://superuser.com/users/432...",278403,0,1607663
1607672,2020-12-06T04:10:54.117,0,<p>I've been searching and the problem is &quo...,1240079,0,1601514


In [215]:
#Not as relevant
comments[['PostId', 'Score', 'Text', 'UserId']]

,PostId,Score,Text,UserId
Id,,,,
32,27,0,I haven't heard of ferrite beads before. Inter...,17
36,37,0,Had the same problem on my white MacBook. It ...,14
44,81,1,"Of course, there's MAMP for Mac OS X as well, ...",55
45,71,3,"nice! Never heard of it, but that is exactly t...",63
46,84,0,"+1 thx, I should have known Wikipedia would ha...",63
...,...,...,...,...
2450635,1607367,0,CPUs get excited when every instruction they h...,139240
2450636,1607675,0,Please post some sample data to help us unders...,427317
2450637,1607647,0,Converted documents are always tricky. They do...,1219214


In [209]:
# postHistory is how post has changed, not relevant
postHistory

,PostHistoryTypeId,PostId,RevisionGUID,CreationDate,UserId,Text,ContentLicense,UserDisplayName,Comment
Id,,,,,,,,,
1,2,1,c53f190a-e2b3-44c6-b264-f7866d095b89,2009-07-15T06:27:46.723,1,A Windows Web Server 2008 virtual machine I us...,CC BY-SA 2.5,NaN,NaN
2,1,1,c53f190a-e2b3-44c6-b264-f7866d095b89,2009-07-15T06:27:46.723,1,"Why does the /winsxs folder grow so large, and...",CC BY-SA 2.5,NaN,NaN
3,3,1,c53f190a-e2b3-44c6-b264-f7866d095b89,2009-07-15T06:27:46.723,1,<windows><winsxs><disk-space>,CC BY-SA 2.5,NaN,NaN
51,2,24,c6fbff7f-6e29-423a-9165-40d3fa12397c,2009-07-08T16:32:47.817,NaN,"Folks,\r\n\r\nI used to record important techn...",CC BY-SA 2.5,essamSALAH,NaN
52,1,24,c6fbff7f-6e29-423a-9165-40d3fa12397c,2009-07-08T16:32:47.817,NaN,Remove noise from a recording,CC BY-SA 2.5,essamSALAH,NaN
...,...,...,...,...,...,...,...,...,...
6026890,2,1607677,5d6e505a-79fc-4d11-8d5e-4cf398ecf538,2020-12-06T05:11:18.143,722192,I have a problem which I cannot figure out. My...,CC BY-SA 4.0,NaN,NaN
6026891,1,1607677,5d6e505a-79fc-4d11-8d5e-4cf398ecf538,2020-12-06T05:11:18.143,722192,Windows 10 no internet on one network,CC BY-SA 4.0,NaN,NaN
6026892,3,1607677,5d6e505a-79fc-4d11-8d5e-4cf398ecf538,2020-12-06T05:11:18.143,722192,<windows-10><networking><wifi-configuration><i...,CC BY-SA 4.0,NaN,NaN


In [ ]:
# users, don't rerun
users = users.drop_duplicates(subset=['AccountId']).reset_index().set_index('AccountId')

In [205]:
users

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl
AccountId,,,,,,,,,,,,
-1,-1,1,2009-07-12T22:51:42.563,Community,2008-07-31T00:00:00.000,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",79,53210,168298,NaN
1,1,22674,2009-07-12T22:51:42.563,Jeff Atwood,2020-12-03T19:13:49.677,http://www.codinghorror.com/blog/,"El Cerrito, CA","<p><a href=""http://www.codinghorror.com/blog/a...",60396,1706,64,NaN
2,2,363,2009-07-12T22:51:42.563,Geoff Dalgas,2020-11-30T21:21:33.480,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the Stack Overflow team. Find...,337,47,1,https://i.stack.imgur.com/nDllk.png
3,3,101,2009-07-12T22:51:42.563,Jarrod Dixon,2020-11-18T18:24:15.340,http://jarroddixon.com,"Raleigh, NC, United States","<p><a href=""http://blog.stackoverflow.com/2009...",235,533,0,NaN
4,4,1774,2009-07-12T22:51:42.563,Joel Spolsky,2020-10-28T20:04:59.157,https://joelonsoftware.com/,"New York, NY","<p>In 2000 I co-founded Fog Creek Software, wh...",1115,81,8,https://i.stack.imgur.com/C5gBG.jpg?s=128&g=1
...,...,...,...,...,...,...,...,...,...,...,...,...
7571261,1248254,1,2020-12-06T04:40:59.623,user5745279,2020-12-06T04:40:59.623,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/5e42ec9baa5bb0...
20143310,1248255,1,2020-12-06T04:56:51.923,aussietheatre71,2020-12-06T04:56:51.923,http://db.aussietheatre.com.au,NaN,"<p>in our website you can find <a href=""http:/...",0,0,0,https://www.gravatar.com/avatar/029938f592e3c8...
20143320,1248256,1,2020-12-06T04:59:35.167,Scooba Dooba,2020-12-06T04:59:35.167,NaN,NaN,NaN,0,0,0,https://lh5.googleusercontent.com/-AVVAs47x6q0...


In [207]:
score

,UpVotes,DownVotes,Reputation,Views
AccountId,,,,
-1,53210,168298,1,79
1,1706,64,22674,60396
2,47,1,363,337
3,533,0,101,235
4,81,8,1774,1115
...,...,...,...,...
7571261,0,0,1,0
20143310,0,0,1,0
20143320,0,0,1,0


In [206]:
score = users[['UpVotes', 'DownVotes', 'Reputation', 'Views']].astype(int)
score.describe()

,UpVotes,DownVotes,Reputation,Views
count,957889.000000,957889.000000,957889.000000,957889.000000
mean,3.454732,0.510287,61.691174,6.256935
std,76.751208,181.633768,791.094495,168.099552
min,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,101.000000,2.000000
max,53210.000000,168298.000000,338633.000000,63301.000000


Because the up votes and down votes are heavily skew to the top users compared to reputation, it is hard ot determine if to use up votes and down votes are to use reputation, however, we also see that the reputation value is not necessarily related to up votes and down votes and may include views, which may not necessarily mean they are better question answerers. We will likely use either up votes and down votes as a metric of the ability of the user, or use reputation, however, we know we have this data for users, and the most relevant users, those who answer questions likely have this data as well that we can use to help build reviews or identify cold start problems (Where users have no votes or reputation) that we can use a different recommendation system weight to address.

In [1]:
#plt.plot(score.UpVotes.values)
plt.plot(score.Reputation.values)

NameError: name 'plt' is not defined

In [197]:
score.UpVotes.value_counts()

0       776650
1        44215
2        27073
3        16409
4        12132
         ...  
854          1
1685         1
662          1
2198         1
740          1
Name: UpVotes, Length: 968, dtype: int64

In [160]:
users.reset_index().set_index('AccountId').loc[users['AccountId'].value_counts()[users['AccountId'].value_counts() > 1].index.tolist()]

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl
AccountId,,,,,,,,,,,,
13532404,1037001,21,2019-05-16T08:17:58.683,Gil Brudner,2020-12-02T08:26:32.703,NaN,NaN,NaN,7,0,0,NaN
13532404,1054370,1,2019-06-25T07:08:40.287,RoboGil,2019-06-25T07:08:40.287,NaN,Israel,NaN,0,0,0,https://lh4.googleusercontent.com/-4W2vAx7TpgQ...
16987074,1114436,1,2019-11-20T16:32:04.150,Brandon,2019-11-20T16:32:04.150,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/17ee0503dfd5a0...
16987074,1114450,1,2019-11-20T17:10:54.867,a25bedc5-3d09-41b8-82fb-ea6c353d75ae,2019-11-20T17:10:54.867,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/17ee0503dfd5a0...
115519,94996,101,2011-08-21T10:47:53.257,Chamika Sandamal,2020-10-28T11:07:09.727,http://cmsnsoftware.blogspot.com/,Sri Lanka,"<p><a href=""http://cmsnsoftware.blogspot.com/""...",5,0,0,NaN
115519,185284,101,2013-01-08T09:04:10.107,Chamika Sandamal,2015-04-02T15:42:31.373,http://tutorials.cmsnsoftware.com/,Sri Lanka,"<h1>SOreadytohelp</h1>\n\n<p><img src=""https:/...",4,1,0,NaN
8813383,616858,33,2016-07-13T17:12:16.010,Aimsworthy,2020-04-21T17:10:24.077,,"Colorado, United States",,14,0,0,NaN
8813383,691141,1,2017-01-28T17:08:18.260,Aimsworthy,2017-01-28T17:08:18.260,NaN,NaN,NaN,1,0,0,https://www.gravatar.com/avatar/de77a540ab8a82...
14613601,1040937,11,2019-05-24T20:25:15.727,alokp,2019-05-26T06:44:09.763,NaN,NaN,NaN,8,0,0,NaN


In [45]:
tags

,TagName,Count,ExcerptPostId,WikiPostId
Id,,,,
203,firmware,425,392824,392823
204,upgrade,1183,320795,320794
205,networking,29881,244736,244735
206,audio,6416,257013,257012
207,video,5289,256215,256214
...,...,...,...,...
20479,gitk,1,NaN,NaN
20480,finereader,1,NaN,NaN
20481,apipa,1,1606069,1606068


In [140]:
#tags
tags['Count'] = tags['Count'].astype(int)
tag_count = tags[['TagName', 'Count']]
tag_count[tag_count['Count'] > 1].sort_values('Count', ascending=False).shape[0]
print('Total number of tags: ' + str(tag_count.shape[0]))
print('Tags with more than 1 appearance: ' + str(tag_count[tag_count['Count'] > 1].sort_values('Count', ascending=False).shape[0]))
print('Tags with more than 10 appearance: ' + str(tag_count[tag_count['Count'] > 10].sort_values('Count', ascending=False).shape[0]))
print('Tags with more than 50 appearance: ' + str(tag_count[tag_count['Count'] > 50].sort_values('Count', ascending=False).shape[0]))
print('Tags with more than 100 appearance: ' + str(tag_count[tag_count['Count'] > 100].sort_values('Count', ascending=False).shape[0]))
print('Tags with more than 500 appearance: ' + str(tag_count[tag_count['Count'] > 500].sort_values('Count', ascending=False).shape[0]))

Total number of tags: 5560
Tags with more than 1 appearance: 5390
Tags with more than 10 appearance: 4031
Tags with more than 50 appearance: 2088
Tags with more than 100 appearance: 1380
Tags with more than 500 appearance: 414


We can definitely use tags, as there are many tags that appear in the SuperUser database, and many of them that appear a couple of times, and a large amount that appear many times throughout the posts that have appeared on the forum. We can use this tag data to be able to associate different posts with one another and determine users who are best to handle posts of a certain type of category to be able to make better recommendations.

In [49]:
postLinks

,CreationDate,PostId,RelatedPostId,LinkTypeId
Id,,,,
4521,2010-04-26T03:54:37.840,218,9325,1
4522,2010-04-26T03:54:37.840,223,1184,1
4540,2010-04-26T03:54:38.137,951,1227,1
4542,2010-04-26T03:54:38.153,987,2986,1
4545,2010-04-26T03:54:38.200,1114,8127,1
...,...,...,...,...
65501190,2020-12-05T20:35:12.920,1607619,1018673,3
65501191,2020-12-05T20:35:12.920,1607619,949569,3
65501333,2020-12-05T23:01:14.217,1607643,879977,1


In [104]:
# Number of related posts per post in PostLinks
pd.Series(postLinks.PostId.value_counts().values).value_counts()

1     48460
2     20131
3      3465
4      1465
5       512
6       198
7        65
8        35
9        17
10        6
11        3
12        2
22        1
17        1
13        1
dtype: int64

In [116]:
posts_with_links = len(set(postLinks.PostId.unique()).union(set(postLinks.RelatedPostId.unique())))
all_posts = len(postHistory.PostId.unique())
print('Total Number of Posts ' + str(all_posts) + ' | Posts with Links: ' + str(posts_with_links))
print('Percentage of Posts with more links ' + str(posts_with_links/all_posts * 100))

Total Number of Posts 1113829 | Posts with Links: 110866
Percentage of Posts with more links 9.953592517343326


We see that there is already data on related posts that are similar to a post, and so we can use this with posts we determine are similar to new questions, but because only 10% of the posts have related links, we still need to be able to compute similarity with NLP using the post text as shown below.

In [117]:
posts.loc[['1607619']].Body.values[0]

"<p>Can anyone help me out? I've searched here on Superuser, I've searched Google, I've searched everywhere, and I cannot for the life of me find out how to shut down Cortana. Mind you, I have a new laptop, so maybe information hasn't caught up yet.</p>\n<p>She's everywhere. She's there when I Alt+Tab. She's there in the corner. I've shut down permissions everywhere I can find. But she's still there. Can anyone tell me once and for all how to shut down Cortana? I shouldn't have to fight with Microsoft just to get Cortana to leave me alone.</p>\n<p>In case it matters, I have Windows 10 build 19041.630.</p>\n"

In [77]:
#Badges are irrelevant to us
badges

,UserId,Name,Date,Class,TagBased
Id,,,,,
1,1,Organizer,2009-07-15T06:51:46.370,3,False
2,3,Organizer,2009-07-15T06:51:46.387,3,False
4,1,Autobiographer,2009-07-15T06:51:46.447,3,False
5,2,Autobiographer,2009-07-15T06:51:46.447,3,False
6,3,Autobiographer,2009-07-15T06:51:46.447,3,False
...,...,...,...,...,...
1787182,287827,Excavator,2020-12-06T04:52:34.833,3,False
1787183,623483,Supporter,2020-12-06T04:52:34.833,3,False
1787184,498071,Teacher,2020-12-06T04:52:34.833,3,False


In [76]:
#Votes are irrelevant to us
votes.VoteTypeId.value_counts()

2     3309255
3      488798
5      335197
10     231539
1      189879
15     105263
16      94318
6       47155
12      25250
8        8820
9        8811
11       5904
4        1476
7         264
Name: VoteTypeId, dtype: int64

# Validation and Test Data

In [10]:
relevant = posts[['PostTypeId', 'CreationDate', 'Score', 'Body', 'Tags', 'OwnerUserId', 'AnswerCount', 'CommentCount', 'ParentId', 'ClosedDate']]
questions = relevant[relevant['PostTypeId'] == '1'].drop(['PostTypeId', 'ParentId'], axis=1)
answers = relevant[relevant['PostTypeId'] == '2'].drop(['PostTypeId', 'AnswerCount', 'ClosedDate', 'Tags'], axis=1)

In [26]:
relevant['CreationDate'] = pd.to_datetime(relevant['CreationDate'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Id
1         2009-07-15 06:27:46.723
24        2009-07-08 16:32:47.817
25        2009-07-08 16:34:52.867
26        2009-07-08 16:59:00.587
27        2009-07-08 17:15:59.477
                    ...          
1607673   2020-12-06 04:16:34.360
1607675   2020-12-06 04:27:04.807
1607676   2020-12-06 04:34:37.490
1607677   2020-12-06 05:11:18.143
1607678   2020-12-06 05:14:36.420
Name: CreationDate, Length: 1113829, dtype: datetime64[ns]

In [18]:
1113829*.2

222765.80000000002

In [83]:
train_data = relevant[relevant['CreationDate'] < pd.datetime(2017, 1, 1)]
evaluation_data = relevant[relevant['CreationDate'] >= pd.datetime(2017, 1, 1)]
shuffled_data = evaluation_data.sample(frac=1).reset_index(drop=False)
validation_data = shuffled_data.loc[int(shuffled_data.shape[0]/2):]
test_data = shuffled_data[:int(shuffled_data.shape[0]/2)]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [77]:
valid_questions = validation_data[validation_data['PostTypeId'] == '1'].drop(['PostTypeId', 'ParentId'], axis=1)
valid_answers = validation_data[validation_data['PostTypeId'] == '2'].drop(['PostTypeId', 'AnswerCount', 'ClosedDate', 'Tags'], axis=1)

In [85]:
test_questions = test_data[test_data['PostTypeId'] == '1'].drop(['PostTypeId', 'ParentId'], axis=1)
test_answers = test_data[test_data['PostTypeId'] == '2'].drop(['PostTypeId', 'AnswerCount', 'ClosedDate', 'Tags'], axis=1)

In [79]:
valid_q = valid_questions[valid_questions.Id.isin(valid_answers.ParentId.values)]
valid_a = valid_answers[valid_answers.ParentId.isin(valid_questions.Id.values)]

In [86]:
test_q = test_questions[test_questions.Id.isin(test_answers.ParentId.values)]
test_a = test_answers[test_answers.ParentId.isin(test_questions.Id.values)]